In [1]:
import torch 
from models.BareboneGCN import BareboneGCN
from CodeGraphDataset_diversevul_erik_preprocessed import CodeGraphDataset_InMemory
dataset4 = CodeGraphDataset_InMemory(pt_folder='codegraphs/diversevul/v2_undirected_withdegreecount', split='train', 
                                          cross_val_valfold_idx=0, 
                                          is_cross_val=True, cross_val_train_fraction=0.01, DS_type='all')

Split 0: 51069
Split 1: 54059
Split 2: 72263
Split 3: 53178
Split 4: 42936


Processing...
Done!


codegraphs/diversevul/v2_undirected_withdegreecount/5ac224602f1d603aac5eaa72e1760d3e33a26f0a_120848610895421645248357269512901044407_0.cpg.pt


100%|██████████| 2222/2222 [00:00<00:00, 2360.88it/s]


cpu


# Test GraphGLOW with DENSE

In [2]:
i = 0
while True:
    sample = dataset4[i]
    i += 1   
    if sample.x.shape[0]>910 and sample.x.shape[0]<1010:
        break 

In [3]:
sample=sample.cuda()

In [4]:
edge_index = sample.edge_index
# make a dense pytorch matrix out of it
from torch_geometric.utils import to_dense_adj
dense = to_dense_adj(edge_index).squeeze(0)
features = sample.x

In [5]:
dense.shape

torch.Size([911, 911])

In [6]:
len(dataset4)

2222

In [7]:

import torch
# import linear from torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, Linear
from torch_geometric.utils import scatter

def squareroot_degree_inverse_undirected(adj):
    row_sum = torch.sum(adj, dim=1)
    row_sum = row_sum.pow(-0.5)
    D_12 = torch.eye(row_sum.shape[0]).to(row_sum.device) * row_sum
    return D_12


class DenseGCNConv(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DenseGCNConv, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.weight = torch.nn.Parameter(torch.Tensor(in_channels, out_channels))
        self.bias = torch.nn.Parameter(torch.Tensor(out_channels))
        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.weight)
        torch.nn.init.zeros_(self.bias)

    def forward(self, x, adj):
        x = torch.matmul(x, self.weight)
        x = torch.matmul(adj, x)
        return x + self.bias


class BareboneDenseGCN(torch.nn.Module):
    def __init__(self, hidden_channels, num_layers, dropout, readout_pooling,in_channels=162, **kwargs):
        super().__init__()
        
        self.dropout = dropout
        self.pooling = readout_pooling
        self.layers = torch.nn.ModuleList()
        
        self.initial_layer = DenseGCNConv(in_channels, hidden_channels)
        
        self.layers = torch.nn.ModuleList()
        for i in range(num_layers-1):
            if i == 0:
                self.layers.append(DenseGCNConv(in_channels, hidden_channels))
            else:
                self.layers.append(DenseGCNConv(hidden_channels, hidden_channels))
            
        # self.classification_layer = Linear(hidden_channels, 1)
        
        
    def forward_initial_layer(self, x, normalized_adj):
        X = self.initial_layer(x, normalized_adj)
        X = F.dropout(X, training=self.training, p=self.dropout) # droput before relu faster
        X = F.relu(X)
        return X

    def forward(self, X, normalized_adj, initial_normalized_adj,GSL_skip_ratio):
        if GSL_skip_ratio is not None and GSL_skip_ratio>0:  # add skip connections
            for layer in self.layers:
                X_skip = layer(X, initial_normalized_adj)
                X_nonskip = layer(X, normalized_adj)
                X = F.relu(GSL_skip_ratio*X_skip + (1-GSL_skip_ratio)*X_nonskip)
                X = F.dropout(X, training=self.training, p=self.dropout)
        else:   
            for layer in self.layers:
                X = layer(X, normalized_adj)
                X = F.dropout(X, training=self.training, p=self.dropout)
                X = F.relu(X)
            
        # logits = self.classification_layer(X)

        return X
    

In [8]:
class StructureLearner(torch.nn.Module):
    def __init__(self, num_heads, input_channels, **kwargs):
        super().__init__()
        self.weight_tensor = torch.nn.Parameter(torch.Tensor(num_heads, input_channels)).unsqueeze(1)
        self.weight_tensor = torch.nn.Parameter(torch.nn.init.xavier_uniform_(self.weight_tensor))
        self.attention_threshold_epsilon = 0.3

    def forward(self, X):
        X = X.unsqueeze(0) 
        
        # hadamard product by broadcasting
        # (h, 1, dim) * (1, n, dim) -> (h, n, dim)
        X = torch.multiply(self.weight_tensor, X)
        Xnorm = F.normalize(X, p=2, dim=-1)
        attention = torch.bmm(Xnorm, Xnorm.transpose(1,2).detach())
        attention = torch.mean(attention, dim=0)
        attention[attention > self.attention_threshold_epsilon] = 0

        return attention

In [9]:
hidden_channels = 128
model = BareboneDenseGCN(hidden_channels=hidden_channels, num_layers=2, dropout=0.5, readout_pooling='mean').cuda()


In [10]:
from models.GCN import GCN
edgeindex_model = GCN(hidden_channels=hidden_channels, num_layers=2, dropout=0.5, readout_pooling='mean').cuda()

In [11]:
sample

Data(x=[911, 162], edge_index=[2, 3490], y=[1])

In [12]:
dense.shape

torch.Size([911, 911])

In [13]:
structureLearner = StructureLearner(num_heads=4, input_channels=hidden_channels).cuda()

In [14]:
sample = sample.cuda()

In [15]:
from models.BinaryClassifierReadOutLayer import BinaryClassifierReadOutLayer
head = BinaryClassifierReadOutLayer(hidden_channels=hidden_channels, num_layers=1, pooling='sum').cuda()

In [16]:
features.shape
batch = torch.zeros(sample.x.shape[0]).long().cuda()

In [24]:
num_samples = 100
batch_size = 8
iterations = 5
model.train()
edgeindex_model.train()
y = torch.tensor([[1.0]]).cuda()
optimizer1 = torch.optim.Adam(model.parameters(), lr=0.001)
optimizer2 = torch.optim.Adam(edgeindex_model.parameters(), lr=0.001)

In [26]:
%%timeit
# run model

for i in range(num_samples):
    
    dense = to_dense_adj(edge_index).squeeze(0)
    normalized_dense_initial = squareroot_degree_inverse_undirected(dense)
    features = sample.x
    h = model.forward_initial_layer(features, normalized_dense_initial)
    for iter in range(iterations):
        adj = structureLearner(h)
        normalized_dense = squareroot_degree_inverse_undirected(adj)
        h = model(features, normalized_dense, normalized_dense_initial, GSL_skip_ratio=0.3) # in graphglow they use same h as the one from the initial layer
    
    logits = head(h, batch)
    loss = F.binary_cross_entropy_with_logits(logits, y)
    
    
    if i % batch_size == 0 and i > 0:
        loss.backward()
        optimizer1.step()
        optimizer1.zero_grad()
    else:
        loss.backward(retain_graph=True)
    
    

1.63 s ± 67.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
from models.GraphStructureLearner import GraphStructureLearner
sparse_gsl = GraphStructureLearner(num_heads=4, input_channels=hidden_channels, attention_threshold_epsilon=0.3).cuda()
has_converged = torch.tensor([False]).cuda()

from models.BinaryClassifierReadOutLayer import BinaryClassifierReadOutLayer
head2 = BinaryClassifierReadOutLayer(hidden_channels=hidden_channels, num_layers=1, pooling='sum').cuda()

In [27]:
%%timeit
# run model

for i in range(num_samples):
    features = sample.x
    h = edgeindex_model.initial_layer(features, sample.edge_index)
    for iter in range(iterations):
        edge_index, edge_weights = sparse_gsl(h, batch, has_converged) # note that this uses the minibatch implementation (iterate over graphs in minibatch seperately), I checked that most time consuming is the bmm and the torch.argwhere, therefore the batch overhead should not be significant
        h, placeholder = edgeindex_model(features, edge_index, edge_weights, sample.edge_index, GSL_skip_ratio=0.3)
    
    logits = head(h, batch)
    loss = F.binary_cross_entropy_with_logits(logits, y)
    
    if i % batch_size == 0 and i > 0:
        loss.backward()
        optimizer2.step()
        optimizer2.zero_grad()
    else:
        loss.backward(retain_graph=True)
        

KeyboardInterrupt: 

: 

In [ ]:
logits.shape

torch.Size([3364, 1])